<a href="https://colab.research.google.com/github/truthofmyrrh/CJ_Demand_Forecasting/blob/main/CODE/%EC%99%B8%EB%B6%80%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%88%98%EC%A7%91.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from urllib.request import urlopen
from urllib.parse import urlencode, unquote, quote_plus
import urllib
import requests
import json

import pandas as pd

In [ ]:
from lxml import html
import requests, bs4

!pip install xmltodict
import xmltodict

# ***기상 정보 수집***

In [ ]:
gisangchung = pd.read_csv('/content/drive/MyDrive/예측 모델링/Data/시도별_기상청정보.csv',encoding='cp949')
gisangchung

,시/도,지점,지점명,관리관서
0,서울특별시,108,서울,수도권기상청
1,부산광역시,159,부산,부산지방기상청
2,대구광역시,143,대구,대구지방기상청
3,인천광역시,112,인천,수도권기상청
4,광주광역시,156,광주,광주지방기상청
5,대전광역시,133,대전,대전지방기상청
6,울산광역시,152,울산,울산기상대
7,세종특별자치시,239,세종,대전지방기상청
8,경기도,119,수원,수도권기상청
9,강원도,104,북강릉,강원지방기상청


In [ ]:
gisang_dic = gisangchung[['시/도','지점']].set_index('지점').T.to_dict('index')
gisang_dic = gisang_dic['시/도']
gisang_dic

{104: '강원도',
 108: '서울특별시',
 112: '인천광역시',
 119: '경기도',
 131: '충북',
 133: '대전광역시',
 136: '경북',
 143: '대구광역시',
 146: '전북',
 152: '울산광역시',
 155: '경남',
 156: '광주광역시',
 159: '부산광역시',
 168: '전남',
 177: '충남',
 184: '제주',
 239: '세종특별자치시'}

In [ ]:
import time

In [ ]:
key = '****************'
url = 'http://apis.data.go.kr/1360000/AsosHourlyInfoService/getWthrDataList'

res_weather = []

for i in range(0,len(gisang_dic)):
    area = str(gisangchung['지점'][i])
    print(gisang_dic[int(area)] + " 데이터 수집 중...")
    for j in range(2,7):
        startDt = "20210" + str(j) + "01"
        endDt = "20210" + str(j+1) + "01"
        
        if j == 2:
            numOfRows = '672'
        else:
            if j%2 == 0 :
                numOfRows = '720'
            else:
                numOfRows = '744'

        #url 설정
        queryParams = '?' + urlencode({ quote_plus('ServiceKey') : key,
                                        quote_plus('numOfRows') : numOfRows,
                                        quote_plus('pageNo') : '1',
                                        quote_plus('dataCd') : 'ASOS', 
                                        quote_plus('dateCd') : 'HR', 
                                        quote_plus('stnIds') : area,
                                        quote_plus('endDt') : endDt,
                                        quote_plus('endHh') : '01',
                                        quote_plus('startHh') : '01', 
                                        quote_plus('startDt') : startDt
                                        })
        try:
            request = requests.get(url + queryParams).content
        except:
            print(url + queryParams)
            time.sleep(1)
            request = requests.get(url + queryParams).content

        r_xml = xmltodict.parse(request)
        dict_data = r_xml['response']['body']['items']

        if dict_data:
            dict_data = dict_data['item']
            if str(type(dict_data)) == "<class 'list'>" :
                for i in range(0,len(dict_data)):
                    res_weather.append(list(dict_data[i].values()))
            else:
                res_weather.append(list(dict_data.values()))
            # print(res_weather)


df_weather = pd.DataFrame(res_weather).loc[:,[0,3,4]]
df_weather.columns=['시간', '지역명', '기온']

서울특별시 데이터 수집 중...
http://apis.data.go.kr/1360000/AsosHourlyInfoService/getWthrDataList?ServiceKey=sNjZTCadAcfeyftEc2tj6A1m2mD0pdjn34g6e8sOPcB0cpNKv8QSJFQnmkeP18u1jHZsDikb4aJ0WPPI7MNH%2Bw%3D%3D&numOfRows=720&pageNo=1&dataCd=ASOS&dateCd=HR&stnIds=108&endDt=20210701&endHh=01&startHh=01&startDt=20210601
부산광역시 데이터 수집 중...
대구광역시 데이터 수집 중...
http://apis.data.go.kr/1360000/AsosHourlyInfoService/getWthrDataList?ServiceKey=sNjZTCadAcfeyftEc2tj6A1m2mD0pdjn34g6e8sOPcB0cpNKv8QSJFQnmkeP18u1jHZsDikb4aJ0WPPI7MNH%2Bw%3D%3D&numOfRows=720&pageNo=1&dataCd=ASOS&dateCd=HR&stnIds=143&endDt=20210501&endHh=01&startHh=01&startDt=20210401
인천광역시 데이터 수집 중...
광주광역시 데이터 수집 중...
대전광역시 데이터 수집 중...
http://apis.data.go.kr/1360000/AsosHourlyInfoService/getWthrDataList?ServiceKey=sNjZTCadAcfeyftEc2tj6A1m2mD0pdjn34g6e8sOPcB0cpNKv8QSJFQnmkeP18u1jHZsDikb4aJ0WPPI7MNH%2Bw%3D%3D&numOfRows=720&pageNo=1&dataCd=ASOS&dateCd=HR&stnIds=133&endDt=20210501&endHh=01&startHh=01&startDt=20210401
울산광역시 데이터 수집 중...
세종특별자치시 데이터 수집 중...
경기도 데

In [ ]:
df_weather

,시간,지역명,기온
0,2021-02-01 01:00,서울,6.3
1,2021-02-01 02:00,서울,6.2
2,2021-02-01 03:00,서울,5.7
3,2021-02-01 04:00,서울,5.7
4,2021-02-01 05:00,서울,5.6
...,...,...,...
61195,2021-06-30 20:00,제주,24.7
61196,2021-06-30 21:00,제주,24.3
61197,2021-06-30 22:00,제주,24.4
61198,2021-06-30 23:00,제주,24.2


In [ ]:
df_weather.reset_index(drop=True,inplace=True)
df_weather

,시간,지역명,기온
0,2021-02-01 01:00,서울,6.3
1,2021-02-01 02:00,서울,6.2
2,2021-02-01 03:00,서울,5.7
3,2021-02-01 04:00,서울,5.7
4,2021-02-01 05:00,서울,5.6
...,...,...,...
61195,2021-06-30 20:00,제주,24.7
61196,2021-06-30 21:00,제주,24.3
61197,2021-06-30 22:00,제주,24.4
61198,2021-06-30 23:00,제주,24.2


In [ ]:
# 형변환
df_weather['시간'] = df_weather['시간'].astype("string")

In [ ]:
dateArr = []
timeArr = []
for i in range(0,len(df_weather)):
    dateArr.append((df_weather['시간'][i].split(' '))[0])
    timstr=(df_weather['시간'][i].split(' '))[1]
    timeArr.append(int(timstr.split(':')[0]))
df_weather['날짜'] = dateArr
df_weather['시간'] = timeArr

In [ ]:
df_weather = df_weather[['날짜','시간','지역명','기온']];df_weather

,날짜,시간,지역명,기온
0,2021-02-01,1,서울,6.3
1,2021-02-01,2,서울,6.2
2,2021-02-01,3,서울,5.7
3,2021-02-01,4,서울,5.7
4,2021-02-01,5,서울,5.6
...,...,...,...,...
61195,2021-06-30,20,제주,24.7
61196,2021-06-30,21,제주,24.3
61197,2021-06-30,22,제주,24.4
61198,2021-06-30,23,제주,24.2


In [ ]:
df_weather.to_csv("/content/drive/MyDrive/예측 모델링/Data/시도별_기상데이터.csv",encoding='cp949')

# ***특일 정보 수집(공휴일)***

In [ ]:
key = '****************'
url = 'http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getHoliDeInfo'

res_hyuil = []

for i in range(2, 7):
    Dt = '0' + str(i)
    print(i, '월 데이터 수집 중...')

    #url 설정
    queryParams = '?' + urlencode({quote_plus('solYear') : '2021',
                                   quote_plus('solMonth') : Dt,
                                   quote_plus('ServiceKey') : key
                                    })
          
    request = requests.get(url + queryParams).content
    r_xml = xmltodict.parse(request)
    dict_data = r_xml['response']['body']['items']
    
    if dict_data:
        dict_data = dict_data['item']
        if str(type(dict_data)) == "<class 'list'>" :
            for i in range(0,len(dict_data)):
                res_hyuil.append(list(dict_data[i].values()))
        else:
            res_hyuil.append(list(dict_data.values()))


df_holiday = pd.DataFrame(res_hyuil).loc[:,[3,1,2]]
df_holiday.columns=['날짜','명칭','휴일여부']

2 월 데이터 수집 중...
3 월 데이터 수집 중...
4 월 데이터 수집 중...
5 월 데이터 수집 중...
6 월 데이터 수집 중...


In [ ]:
df_holiday

,날짜,명칭,휴일여부
0,20210211,설날,Y
1,20210212,설날,Y
2,20210213,설날,Y
3,20210301,삼일절,Y
4,20210505,어린이날,Y
5,20210519,부처님오신날,Y
6,20210606,현충일,Y


In [ ]:
df_holiday['날짜'] = df_holiday['날짜'].astype("string")

In [ ]:
from datetime import datetime
format = '%Y-%m-%d'
dateArr = []
for i in range(0,len(df_holiday)) :
    date = datetime.strptime(df_holiday['날짜'][i], '%Y%m%d')
    str_date = datetime.strftime(date,format)
    dateArr.append(str_date)
df_holiday['날짜'] = dateArr

In [ ]:
df_holiday

,날짜,명칭,휴일여부
0,2021-02-11,설날,Y
1,2021-02-12,설날,Y
2,2021-02-13,설날,Y
3,2021-03-01,삼일절,Y
4,2021-05-05,어린이날,Y
5,2021-05-19,부처님오신날,Y
6,2021-06-06,현충일,Y


In [ ]:
df_holiday.to_csv("/content/drive/MyDrive/예측 모델링/Data/공휴일데이터.csv",encoding='cp949')

# ***코로나 시,도 발생현황***

In [ ]:
key = '***************'
url = 'http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19SidoInfStateJson'

#url 설정
queryParams = '?' + urlencode({quote_plus('ServiceKey') : key,
                               quote_plus('pageNo') : '1',
                               quote_plus('numOfRows') : '100',
                               quote_plus('startCreateDt') : '20210201',
                               quote_plus('endCreateDt') : '20210630'
                               })
        

request = requests.get(url + queryParams).content
r_xml = xmltodict.parse(request)
dict_data = r_xml['response']['body']['items']['item']

df = pd.DataFrame(dict_data)
df_corona = df[['createDt', 'gubun','localOccCnt']]
df_corona.columns = ['기준일시', '지역', '지역발생수']
df_corona

,기준일시,지역,지역발생수
0,2021-06-30 09:00:00.000,합계,759
1,2021-06-30 09:00:00.000,서울,368
2,2021-06-30 09:00:00.000,부산,13
3,2021-06-30 09:00:00.000,대구,11
4,2021-06-30 09:00:00.000,인천,29
...,...,...,...
2845,2021-02-01 09:00:00.000,전남,0
2846,2021-02-01 09:00:00.000,경북,4
2847,2021-02-01 09:00:00.000,경남,4
2848,2021-02-01 09:00:00.000,제주,0


In [ ]:
df_corona['기준일시'] = df_corona['기준일시'].astype("string")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
dateArr = []
for i in range(0,len(df_corona)):
    dateArr.append((df_corona['기준일시'][i].split(' '))[0])
df_corona['기준일시'] = dateArr
df_corona

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,기준일시,지역,지역발생수
0,2021-06-30,합계,759
1,2021-06-30,서울,368
2,2021-06-30,부산,13
3,2021-06-30,대구,11
4,2021-06-30,인천,29
...,...,...,...
2845,2021-02-01,전남,0
2846,2021-02-01,경북,4
2847,2021-02-01,경남,4
2848,2021-02-01,제주,0


In [ ]:
df_corona.to_csv("/content/drive/MyDrive/예측 모델링/Data/시도별_코로나데이터.csv",encoding='cp949')